# Quantization aware training in Keras example

## Overview

Welcome to an end-to-end example for *quantization aware training*.

### Other pages
For an introduction to what quantization aware training is and to determine if you should use it (including what's supported), see the [overview page](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).

To quickly find the APIs you need for your use case (beyond fully-quantizing a model with 8-bits), see the
[comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md).

### Summary

In this tutorial, you will:

1.   Train a `tf.keras` model for MNIST from scratch.
2.   Fine tune the model by applying the quantization aware training API, see the accuracy, and
     export a quantization aware model.
3.   Use the model to create an actually quantized model for the TFLite
     backend.
4.   See the persistence of accuracy in
     TFLite and a 4x smaller model. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

## Setup

In [ ]:
# ! pip install -q tensorflow
# ! pip install -q tensorflow-model-optimization


In [1]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

## Train a model for MNIST without quantization aware training

In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.summary()

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)

model.save("tf_model.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

In [10]:
model.get_weights()

[array([[[[-0.45643595,  0.2089    ,  0.29132396,  0.7581083 ,
            0.17155807, -0.34782812,  0.0736233 , -0.71295077,
            0.13690768, -0.41616172,  0.12428223,  0.2376009 ]],
 
         [[-0.11969946,  0.44221583,  0.23892365, -0.07182363,
            0.25695345,  0.22321887,  0.10940634, -0.6417914 ,
            0.14003332, -0.16148347,  0.31140175,  0.31671897]],
 
         [[ 0.1991883 ,  0.5577426 , -0.09308445, -0.55222845,
           -0.02632828,  0.20184901,  0.14755766, -0.45000005,
           -0.01966464,  0.41319415,  0.1916378 , -0.25791535]]],
 
 
        [[[-0.13957845,  0.23353545,  0.22361813,  0.03748423,
            0.3175359 , -0.4381458 ,  0.13259628,  0.0812076 ,
            0.01900117, -0.40254873,  0.285555  ,  0.06350007]],
 
         [[ 0.07650129,  0.21031287, -0.03805352, -1.0033306 ,
            0.2861028 , -0.11202397,  0.2521077 ,  0.2682261 ,
            0.20861635, -0.07898667, -0.03679642,  0.02125458]],
 
         [[ 0.35245678,  0.19230

In [3]:
model.get_weights()[0].dtype

dtype('float32')

## Clone and fine-tune pre-trained model with quantization aware training


### Define the model

You will apply quantization aware training to the whole model and see this in the model summary. All layers are now prefixed by "quant".

Note that the resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8). The sections after show how to create a quantized model from the quantization aware one.

In the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md), you can see how to quantize some layers for model accuracy improvements.

In [4]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

q_aware_model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28)           3         
 er)                                                             
                                                                 
 quant_reshape (QuantizeWrap  (None, 28, 28, 1)        1         
 perV2)                                                          
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 26, 26, 12)       147       
 erV2)                                                           
                                                                 
 quant_max_pooling2d (Quanti  (None, 13, 13, 12)       1         
 zeWrapperV2)                                                    
                                                                 
 quant_flatten (QuantizeWrap  (None, 2028)             1

In [5]:
q_aware_model.get_weights()[0].dtype

dtype('float32')

In [8]:
q_aware_model.get_weights()

[0.0,
 1.0,
 -1,
 -1,
 array([[[[-0.51718473,  0.24837793,  0.25961477,  0.9694432 ,
            0.08281811, -0.3752423 ,  0.08553067, -0.9541065 ,
            0.03805856, -0.5703281 ,  0.10953508,  0.22723937]],
 
         [[-0.17616847,  0.55188763,  0.21173711, -0.04820147,
            0.19499797,  0.28354636,  0.1086151 , -0.8290946 ,
            0.07159813, -0.22359979,  0.30912206,  0.26699123]],
 
         [[ 0.20352145,  0.6717498 , -0.19214942, -0.68250835,
           -0.05543458,  0.2532837 ,  0.05855025, -0.6383301 ,
           -0.04199731,  0.46623805,  0.17037733, -0.48012108]]],
 
 
        [[[-0.19410239,  0.22725362,  0.18127665,  0.02171465,
            0.26567283, -0.57478637,  0.1401248 ,  0.12441295,
           -0.02790124, -0.5608845 ,  0.29290462,  0.09859674]],
 
         [[ 0.1147846 ,  0.26633185, -0.04135625, -1.4320173 ,
            0.2780914 , -0.10750402,  0.2711366 ,  0.31726724,
            0.21837588, -0.08580681, -0.02779168, -0.00805498]],
 
         [

In [11]:
q_aware_model.save("q_aware_model.h5")

### Train and evaluate the model against baseline

To demonstrate fine tuning after training the model for just an epoch, fine tune with quantization aware training on a subset of the training data.

In [12]:
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

2/2 [==============================] - 0s 118ms/step - loss: 0.0830 - accuracy: 0.9756 - val_loss: 0.1065 - val_accuracy: 0.9700


For this example, there is minimal to no loss in test accuracy after quantization aware training, compared to the baseline.

In [13]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9671000242233276
Quant test accuracy: 0.9732000231742859


## Create quantized model for TFLite backend

After this, you have an actually quantized model with int8 weights and uint8 activations.

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Sheena\AppData\Local\Temp\tmptk3ayvlq\assets


INFO:tensorflow:Assets written to: C:\Users\Sheena\AppData\Local\Temp\tmptk3ayvlq\assets
C:\Users\Sheena\anaconda3\envs\q\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [17]:

interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:")
print(input_details)

print("\nOutput details:")
print(output_details)

Input details:
[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([ 1, 28, 28]), 'shape_signature': array([-1, 28, 28]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Output details:
[{'name': 'StatefulPartitionedCall:0', 'index': 19, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [18]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

You evaluate the quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.

In [19]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9732
Quant TF test accuracy: 0.9732000231742859


## See 4x smaller model from quantization

You create a float TFLite model and then see that the quantized TFLite model
is 4x smaller.

In [20]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: C:\Users\Sheena\AppData\Local\Temp\tmpx4ssk58r\assets


INFO:tensorflow:Assets written to: C:\Users\Sheena\AppData\Local\Temp\tmpx4ssk58r\assets


Float model in Mb: 0.08089065551757812
Quantized model in Mb: 0.0238037109375


## Conclusion

In this tutorial, you saw how to create quantization aware models with the TensorFlow Model Optimization Toolkit API and then quantized models for the TFLite backend.

You saw a 4x model size compression benefit for a model for MNIST, with minimal accuracy
difference. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

We encourage you to try this new capability, which can be particularly important for deployment in resource-constrained environments. 
